In [1]:
import math

with open('day20_input.txt') as f:
   lines = [line.strip() for line in f.readlines()]

In [44]:
def split_list(l, val):
    result = []
    current = []
    for item in l:
        if item == val:
            result.append(current)
            current = []
        else:
            current.append(item)
    if current:
        result.append(current)
    return result

def parse_horizontal_line(lines, index):
    result = 0
    multiplier = 1
    for cell in lines[index][::-1]:
        if cell == '#':
            result += multiplier
        multiplier *= 2
    return result

def parse_vertical_line(lines, index):
    result = 0
    multiplier = 1
    reversed_lines = lines[::-1]
    for line in reversed_lines:
        if line[index] == '#':
            result += multiplier
        multiplier *= 2
    return result

def invert(num, length):
    bin_str = '{0:b}'.format(num).rjust(length, '0')
    return int(bin_str[::-1], 2)

class Tile:
    
    def __init__(self, lines=None):
        if lines:
            self.id = int(lines[0][lines[0].index(' '):-1])
            real_lines = lines[1:]
            self.real_lines = real_lines
            self.rotate_count = 0
            self.is_fliph = False
            self.is_flipv = False
            self.size = len(real_lines)
            self.top = parse_horizontal_line(real_lines, 0)
            self.top_inv = invert(self.top, self.size)
            self.bottom = parse_horizontal_line(real_lines, len(real_lines) - 1)
            self.bottom_inv = invert(self.bottom, self.size)
            self.left = parse_vertical_line(real_lines, 0)
            self.left_inv = invert(self.left, self.size)
            self.right = parse_vertical_line(real_lines, len(real_lines) - 1)
            self.right_inv = invert(self.right, self.size)
            print(self.id)
            print(f'{self.top} {self.top_inv} {self.bottom} {self.bottom_inv} {self.left} {self.left_inv} {self.right} {self.right_inv}')
            print()
        
    def copy_to(self, dest, x_off, y_off):
        size = self.size
        if self.rotate_count == 0:
            first_step = [1, 0]
            second_step = [0, 1]
            steps = [first_step, second_step]
        elif self.rotate_count == 1:
            first_step = [0, -1]
            second_step = [1, 0]
            steps = [second_step, first_step]
        elif self.rotate_count == 2:
            first_step = [-1, 0]
            second_step = [0, -1]
            steps = [first_step, second_step]
        else:
            first_step = [0, 1]
            second_step = [-1, 0]
            steps = [second_step, first_step]
        if self.is_fliph:
            steps[0][0] *= -1
        if self.is_flipv:
            steps[1][1] *= -1
        if steps[0][0] < 0:
            x = size - 1
        else:
            x = 0
        if steps[1][1] < 0:
            y = size - 1
        else:
            y = 0
        start_x = x
        start_y = y
        for i in range(size):
            for j in range(size):
                dest[i + y_off][j + x_off] = self.real_lines[y][x]
                y += first_step[1]
                x += first_step[0]
            if first_step[0] == 0:
                y = start_y
            else:
                x = start_x
            y += second_step[1]
            x += second_step[0]
    
    def rotate(self):
        result = Tile()
        result.real_lines = self.real_lines
        result.rotate_count = self.rotate_count + 1
        result.is_fliph = self.is_fliph
        result.is_flipv = self.is_flipv
        result.size = self.size
        result.id = self.id
        result.right = self.top
        result.bottom = self.right_inv
        result.left = self.bottom
        result.top = self.left_inv
        result.right_inv = self.top_inv
        result.bottom_inv = self.right
        result.left_inv = self.bottom_inv
        result.top_inv = self.left
        return result
        
    def fliph(self):
        result = Tile()
        result.real_lines = self.real_lines
        result.rotate_count = self.rotate_count
        result.is_fliph = not self.is_fliph
        result.is_flipv = self.is_flipv
        result.size = self.size
        result.id = self.id
        result.right = self.left
        result.bottom = self.bottom_inv
        result.left = self.right
        result.top = self.top_inv
        result.right_inv = self.left_inv
        result.bottom_inv = self.bottom
        result.left_inv = self.right_inv
        result.top_inv = self.top
        return result
        
    def flipv(self):
        result = Tile()
        result.real_lines = self.real_lines
        result.rotate_count = self.rotate_count
        result.is_fliph = self.is_fliph
        result.is_flipv = not self.is_flipv
        result.size = self.size
        result.id = self.id
        result.right = self.right_inv
        result.bottom = self.top
        result.left = self.left_inv
        result.top = self.bottom
        result.right_inv = self.right
        result.bottom_inv = self.top_inv
        result.left_inv = self.left
        result.top_inv = self.bottom_inv
        return result
    
    def borders(self):
        yield self.left
        yield self.right
        yield self.bottom
        yield self.top
        yield self.left_inv
        yield self.top_inv
        yield self.right_inv
        yield self.bottom_inv
    
    def rotated_incarnations(self):
        yield self
        n = self.rotate()
        yield n
        n = n.rotate()
        yield n
        n= n.rotate()
        yield n        
    
    def incarnations(self):
        for rotation in self.rotated_incarnations():
            yield rotation
        tile = self.fliph()
        for rotation in tile.rotated_incarnations():
            yield rotation
        tile = self.flipv()
        for rotation in tile.rotated_incarnations():
            yield rotation
        tile = self.fliph().flipv()
        for rotation in tile.rotated_incarnations():
            yield rotation
            
    def __repr__(self):
        result = f'Tile {self.id}:\n'
        left = '{0:b}'.format(self.left).rjust(self.size, '0').replace('0', '.').replace('1', '#')
        right = '{0:b}'.format(self.right).rjust(self.size, '0').replace('0', '.').replace('1', '#')
        result += '{0:b}\n'.format(self.top).rjust(self.size, '0').replace('0', '.').replace('1', '#')
        fill = ' ' * (self.size - 2)
        for i in range(self.size - 2):
            result += left[i+1] + fill + right[i+1] + '\n'
        result += '{0:b}\n'.format(self.bottom).rjust(self.size, '0').replace('0', '.').replace('1', '#')
        return result

    def __eq__(self, other):
        return self.id == other.id

    def __hash__(self):
        return hash(self.id)
        
tile_strs = split_list(lines, '')
tiles = [Tile(tile_str) for tile_str in tile_strs]
tile = tiles[0]

3571
816 51 758 445 655 965 322 266

2687
615 921 171 852 574 497 739 797

3049
855 939 657 549 639 1017 827 883

1597
920 103 23 928 878 475 389 646

1301
727 941 575 1009 841 587 879 987

3259
250 380 553 593 133 644 281 610

3989
591 969 507 894 660 165 647 901

3803
753 573 529 545 597 681 581 649

2897
309 690 195 780 192 12 1021 767

2843
883 827 915 807 985 623 523 833

3023
314 370 30 480 44 208 294 402

2887
536 97 754 317 979 815 92 232

1013
634 377 725 685 977 559 351 1002

3733
949 695 649 581 647 901 715 845

3691
399 966 962 271 493 734 748 221

3061
772 131 164 148 628 185 412 230

1489
896 7 437 694 624 57 53 688

1637
1022 511 307 818 716 205 255 1020

1069
655 965 986 367 795 867 944 55

1039
639 1017 658 293 837 651 550 401

2767
331 842 46 464 212 172 872 91

2957
991 1007 406 422 532 161 656 37

2789
405 678 995 799 477 750 911 967

2837
17 544 880 59 203 844 780 195

3251
504 126 844 203 117 696 118 440

3137
1001 607 373 698 964 143 563 817

3433
414 486 16 32 1

In [45]:
print(tile)
outgrid = []
for _ in range(20):
    rowgrid = []
    for _ in range(20):
        rowgrid.append(' ')
    outgrid.append(rowgrid)

tile.fliph().rotate().copy_to(outgrid, 3, 3)

def print_grid(grid):
    for j in range(len(grid)):
        row = grid[j]
        for i in range(len(row)):
            print(row[i], end='')
        print()
        
print_grid(outgrid)

Tile 3571:
##..##....
.        #
#        .
.        #
.        .
.        .
#        .
#        .
#        #
#.####.##.

                    
                    
                    
   .#....#.#.       
   #.##......       
   #..#......       
   ........#.       
   ##..#.##.#       
   ###.#..#.#       
   #...#...#.       
   #.......#.       
   .........#       
   ####...#.#       
                    
                    
                    
                    
                    
                    
                    


In [46]:
border_choices = {}

def record(value, tile):
    if value == 0:
        print("HUH?")
    if value in border_choices:
        border_choices[value].append(tile)
    else:
        border_choices[value] = [tile]

for tile in tiles:
    record(tile.top, tile)
    record(tile.left, tile)
    record(tile.right, tile)
    record(tile.bottom, tile)
    record(tile.top_inv, tile)
    record(tile.left_inv, tile)
    record(tile.right_inv, tile)
    record(tile.bottom_inv, tile)

border_choices

{816: [Tile 3571:
  ##..##....
  .        #
  #        .
  .        #
  .        .
  .        .
  #        .
  #        .
  #        #
  #.####.##.,
  Tile 1487:
  #..#....##
  .        #
  #        .
  #        .
  #        #
  #        #
  .        .
  .        .
  .        .
  #...#..#.],
 655: [Tile 3571:
  ##..##....
  .        #
  #        .
  .        #
  .        .
  .        .
  #        .
  #        .
  #        #
  #.####.##.,
  Tile 1069:
  #.#...####
  #        #
  .        #
  .        .
  .        #
  #        #
  #        .
  .        .
  #        .
  ####.##.#.],
 322: [Tile 3571:
  ##..##....
  .        #
  #        .
  .        #
  .        .
  .        .
  #        .
  #        .
  #        #
  #.####.##.,
  Tile 1747:
  ...##.#..
  #        #
  .        .
  .        .
  #        .
  #        .
  .        #
  #        .
  .        #
  .#####...],
 758: [Tile 3571:
  ##..##....
  .        #
  #        .
  .        #
  .        .
  .        .
  #        .
  #        .

In [47]:
for key, value in border_choices.items():
    print(str(key).ljust(5) + ' ' + str(len(value)))

816   2
655   2
322   2
758   2
51    2
965   2
266   2
445   2
615   2
574   2
739   2
171   2
921   2
497   2
797   2
852   2
855   2
639   2
827   2
657   2
939   2
1017  2
883   2
549   2
920   2
878   2
389   2
23    2
103   2
475   2
646   2
928   2
727   2
841   2
879   2
575   2
941   2
587   2
987   2
1009  2
250   2
133   2
281   2
553   2
380   2
644   2
610   2
593   2
591   2
660   1
647   2
507   2
969   2
165   1
901   2
894   2
753   2
597   2
581   2
529   2
573   2
681   2
649   2
545   2
309   2
192   2
1021  2
195   2
690   2
12    2
767   2
780   2
985   2
523   2
915   2
623   2
833   2
807   2
314   2
44    1
294   2
30    2
370   2
208   1
402   2
480   2
536   2
979   2
92    2
754   2
97    2
815   2
232   2
317   2
634   2
977   2
351   2
725   2
377   2
559   2
1002  2
685   2
949   2
715   2
695   2
845   2
399   2
493   2
748   2
962   2
966   2
734   2
221   2
271   2
772   2
628   2
412   2
164   2
131   2
185   2
230   2
148   2
896   2
624   2
53    2


In [48]:
unmatched_borders = set()
for key, value in border_choices.items():
    if len(value) == 1:
        unmatched_borders.add(key)
unmatched_borders

{9,
 16,
 29,
 32,
 41,
 44,
 47,
 76,
 83,
 88,
 89,
 91,
 101,
 104,
 122,
 130,
 138,
 165,
 198,
 200,
 208,
 211,
 231,
 260,
 289,
 297,
 302,
 324,
 357,
 358,
 376,
 396,
 410,
 423,
 431,
 437,
 439,
 455,
 461,
 466,
 473,
 481,
 499,
 503,
 517,
 530,
 542,
 543,
 563,
 576,
 592,
 594,
 616,
 622,
 641,
 659,
 660,
 664,
 666,
 669,
 679,
 694,
 711,
 718,
 723,
 733,
 736,
 741,
 749,
 751,
 787,
 803,
 805,
 808,
 812,
 813,
 817,
 830,
 872,
 909,
 910,
 911,
 917,
 918,
 919,
 924,
 935,
 950,
 958,
 959,
 967,
 976,
 982,
 989,
 993,
 1015}

In [49]:
def measure_connectivity_one(tile):
    result = 4
    if tile.top in unmatched_borders:
        result -= 1
    if tile.right in unmatched_borders:
        result -= 1
    if tile.bottom in unmatched_borders:
        result -= 1
    if tile.left in unmatched_borders:
        result -= 1
    return result

def measure_connectivity(tile):
    return max(
        measure_connectivity_one(tile),        
        measure_connectivity_one(tile.rotate()),
        measure_connectivity_one(tile.rotate().rotate()),
        measure_connectivity_one(tile.rotate().rotate().rotate()),
        measure_connectivity_one(tile.fliph()),        
        measure_connectivity_one(tile.fliph().rotate()),
        measure_connectivity_one(tile.fliph().rotate().rotate()),
        measure_connectivity_one(tile.fliph().rotate().rotate().rotate()),
        measure_connectivity_one(tile.flipv()),        
        measure_connectivity_one(tile.flipv().rotate()),
        measure_connectivity_one(tile.flipv().rotate().rotate()),
        measure_connectivity_one(tile.flipv().rotate().rotate().rotate()),
        measure_connectivity_one(tile.fliph().flipv()),        
        measure_connectivity_one(tile.fliph().flipv().rotate()),
        measure_connectivity_one(tile.fliph().flipv().rotate().rotate()),
        measure_connectivity_one(tile.fliph().flipv().rotate().rotate().rotate()),
    )

starting_corner = None
for tile in tiles:
    if measure_connectivity(tile) == 2:
        print(str(tile.id).ljust(5) + ' ' + str(measure_connectivity(tile)))
        starting_corner = tile

3881  2
1153  2
1163  2
1619  2


In [50]:
print(tiles[0].id)
print('{0:b}'.format(tiles[0].top).rjust(10, '0'))
print('{0:b}'.format(tiles[0].left).rjust(10, '0'))
print('{0:b}'.format(tiles[0].left_inv).rjust(10, '0'))

with open('C:\\users\\westo\\downloads\\rotated.txt', 'w') as f:
    for tile in tiles:
        f.write(str(tile.rotate()))
        f.write('\n\n')


3571
1100110000
1010001111
1111000101


In [51]:
starting_corner

Tile 1619:
#.....#.#.
.        #
#        .
#        .
#        #
#        .
.        .
#        .
#        .
#.##...###

In [52]:
measure_connectivity(starting_corner)

2

In [53]:
print(starting_corner.top in unmatched_borders)
print(starting_corner.right in unmatched_borders)
print(starting_corner.bottom in unmatched_borders)
print(starting_corner.left in unmatched_borders)
starting_corner = starting_corner.rotate().rotate()
starting_corner

False
True
True
False


Tile 1619:
###...##.#
.        #
.        #
.        .
.        #
#        #
.        #
.        #
#        .
#.#.....#

In [54]:
grid_size = int(math.sqrt(len(tiles)))
unplaced_tiles = tiles.copy()

def find_tile_with_border(val):
    for tile in unplaced_tiles:
        for border in tile.borders():
            if border == val:
                unplaced_tiles.remove(tile)
                return tile
    raise Exception('Could not find tile with val ' + str(val))
    
def rotate_until(tile, l):
    for incarnation in tile.incarnations():
        if l(incarnation):
            return incarnation
    raise Exception('Could not rotate_until')
    
def find_and_rotate_so_left_is(val):
    tile = find_tile_with_border(val)
    return rotate_until(tile, lambda x: x.left == val)

def find_and_rotate_so_top_is(val):
    tile = find_tile_with_border(val)
    return rotate_until(tile, lambda x: x.top == val)

placed_grid = []
for _ in range(grid_size):
    row = []
    for _ in range(grid_size):
        row.append(None)
    placed_grid.append(row)
    
placed_grid[0][0] = starting_corner
unplaced_tiles.remove(starting_corner)
for row in range(grid_size):
    for column in range(1, grid_size):
        current = placed_grid[row][column-1]
        next_tile = find_and_rotate_so_left_is(current.right)
        placed_grid[row][column] = next_tile
    if row < grid_size - 1:
        current_left = placed_grid[row][0]
        next_tile = find_and_rotate_so_top_is(current_left.bottom)
        placed_grid[row+1][0] = next_tile

In [82]:
tile_size = placed_grid[0][0].size
big_grid_size = tile_size * grid_size
big_grid = []
for _ in range(big_grid_size):
    row = []
    for _ in range(big_grid_size):
        row.append(' ')
    big_grid.append(row)

for row_index in range(len(placed_grid)):
    row = placed_grid[row_index]
    for col_index in range(len(row)):
        tile = row[col_index]
        tile.copy_to(big_grid, col_index * tile_size, row_index * tile_size)
        
print_grid(big_grid)

###...##.##..#.#.....#.##..#.####.#..####.##.#..######.#.##....#.##.####...#..###.###.######..#.##...###.#..#..##.#..###
.........##.##.....##..........#..###..###.......###.......##.##.#...##...#....##..#........##..#.........#..###.......#
.##......##........##....#...##..#..........#...#..#....#..##.............#...#.....##.#.##.....#..##..##...####........
..#.#...#..............................##..................##...###..##........##..#.....##.....#..##....#..............
...#.##..##......#...##...#.....#...##.....#....#........##....##.##....#..#..#....#.....###............#....##.......##
#.#...#..##..#.....##.....#.......#.##.##........###...#...##..#.#..#..#.##.###....####.###..........#.......##.#.###...
.....#...##...##..#...#..........##.#..###...........#..#.#......#.#.##.#......##....#.............##..#.#.#....##.#...#
...#.....##.#..#.#.##.............#...###..#....###..#..#.....#..##.....##..#.###.....#..###.....#....#.................
#..##.#............##.......###.

#..#.....###............#....###....##.##..#####....###...........##......##......#....#.....#..........#....##.........
.#.#...##..###..#..###.###.#...##..#..####.#.#######....##....##.##.#......##....##....#....#.#....##.#.....#...##.#..#.
.#.#...##..###..#..###.###.#...##..#..####.#.#######....##....##.##.#......##....##....#....#.#....##.#.....#...##.#..#.
.#.#..............###.##.....##..#.....##.#.#..#.....#....#.....#..#................#.#..###.......##..#..#......#.#...#
##.#.........###...##.......#..#..........#...#.###..##......###............#..........#####.......##.#........#...#..##
#.......#...#...#..###....#.....##..........#....##...##........#....##...#....##....#.#....#.####.##..#.#.............#
#..#..#..###..#.##....#..........#...#.##......#.###....#..........#...........###.#...........##......#..............#.
..####.##..#...###.##..###..#....#.....##.....#..###.#..#....#..#..##...#........#..#....##..#.....##..........#........
.#...........#.....##.#.#..####.

In [83]:
counter = 0
rows_to_remove = []
while counter < big_grid_size:
    rows_to_remove.append(counter)
    counter += 9
    rows_to_remove.append(counter)
    counter += 1
rows_to_remove = rows_to_remove[::-1]
for ix in rows_to_remove:
    del big_grid[ix]
for row in big_grid:
    for ix in rows_to_remove:
        del row[ix]
print_grid(big_grid)

.........##.............#..###..#.......#........##.#......#......#......##..#.......#..#.......
##..................#.....#........#...##....#.............#...#...##.#......#....##...##.......
.#.#...#...........................................###............#..........#......#...........
..#.##........#.##...#...#...##...#....#......##..##.##..#..#..#..#.....#..........#...........#
.#...#....#..........#.....#.##.........#...#.....#.#..##.##.###..####.#........#........#.###..
....#......##..#.#........##.#..#.........#..#.#....#.#..#..........#.............#.#.#..##.#...
..#......#..#.#............#...#..#....#..#..#...#..##...##..#.#.....#..#.....#..#..............
..##.#.................#..#..##.#..#..#..#..#....#..#..###.#...............#....#.#.#.....#.....
......#.....#....#..#...#...#....##....##..#.###........#..##...............................#..#
....#.............#......##....#.......#..#...##..#....#.#.##..#............#...###........#.#..
.##....#...........#...#....#.

.........#........#...#.#..###...###.#.###.......#.....##..#...#..#.#............#.##......#.#..
###..#..#.#.....#.#.#..##....#...#......##.........#........#................#......###.........
....#....#.#....#....#.#......##..##...###.#..#.......##...#....#.###.##.#.#####..#......##....#
.#....#...#..................##.....#.....#..#.####...#........#...###..####....###.##.....###.#
#.##.#..........#........#.#..#....#.......#..#........##..##.#...#.#...#.#.#....#..#.....#...#.
#...#.#...........##..#...##.............#.#....#.......#.......................................


In [84]:
print(len(big_grid))
print(len(big_grid[0]))
gridwidth = 96
gridheight = 96

96
96


In [85]:
sea_monster = [
    '                  # ',
    '#    ##    ##    ###',
    ' #  #  #  #  #  #   '
]
sea_monster_width = len(sea_monster[0])
sea_monster_height = len(sea_monster)
print(f'Here be a sea monster of size {sea_monster_width} x {sea_monster_height}')

Here be a sea monster of size 20 x 3


In [86]:
def is_sea_monster(grid, x_off, y_off):
    for x in range(sea_monster_width):
        for y in range(sea_monster_height):
            if sea_monster[y][x] == '#' and grid[y_off+y][x_off+x] != '#':
                return False
    return True

def has_sea_monster(grid):
    sea_monsters = [
        
    ]
    for x_start in range(gridwidth - sea_monster_width):
        for y_start in range(gridheight - sea_monster_height):
            if is_sea_monster(grid, x_start, y_start):
                sea_monsters.append((x_start, y_start))
    return sea_monsters

print(has_sea_monster(big_grid))
print_grid(big_grid)

[]
.........##.............#..###..#.......#........##.#......#......#......##..#.......#..#.......
##..................#.....#........#...##....#.............#...#...##.#......#....##...##.......
.#.#...#...........................................###............#..........#......#...........
..#.##........#.##...#...#...##...#....#......##..##.##..#..#..#..#.....#..........#...........#
.#...#....#..........#.....#.##.........#...#.....#.#..##.##.###..####.#........#........#.###..
....#......##..#.#........##.#..#.........#..#.#....#.#..#..........#.............#.#.#..##.#...
..#......#..#.#............#...#..#....#..#..#...#..##...##..#.#.....#..#.....#..#..............
..##.#.................#..#..##.#..#..#..#..#....#..#..###.#...............#....#.#.#.....#.....
......#.....#....#..#...#...#....##....##..#.###........#..##...............................#..#
....#.............#......##....#.......#..#...##..#....#.#.##..#............#...###........#.#..
.##....#...........#...#...

In [87]:
def rotate_grid(grid):
    result = []
    for _ in range(gridwidth):
        row = []
        for _ in range(gridwidth):
            row.append(None)
        result.append(row)
    dest_col_index = 0
    dest_row_index = 0
    for row_index in reversed(range(gridwidth)):
        for col_index in range(gridwidth):
            result[dest_row_index][dest_col_index] = grid[row_index][col_index]
            dest_col_index += 1
        dest_col_index = 0
        dest_row_index += 1
    return result

big_grid = rotate_grid(big_grid)
print(has_sea_monster(big_grid))
big_grid = rotate_grid(big_grid)
print(has_sea_monster(big_grid))
big_grid = rotate_grid(big_grid)
print(has_sea_monster(big_grid))

[(1, 26), (2, 44), (6, 14), (15, 58), (20, 81), (22, 4), (22, 50), (24, 31), (27, 75), (28, 63), (28, 86), (33, 19), (41, 25), (42, 40), (48, 14), (50, 90), (54, 56), (58, 67), (63, 1), (63, 78), (64, 36), (67, 51), (70, 41), (73, 8)]
[]
[(1, 26), (2, 44), (6, 14), (15, 58), (20, 81), (22, 4), (22, 50), (24, 31), (27, 75), (28, 63), (28, 86), (33, 19), (41, 25), (42, 40), (48, 14), (50, 90), (54, 56), (58, 67), (63, 1), (63, 78), (64, 36), (67, 51), (70, 41), (73, 8)]


In [80]:
x = """
.#.#..#.##...#.##..#####
###....#.#....#..#......
##.##.###.#.#..######...
###.#####...#.#####.#..#
##.#....#.##.####...#.##
...########.#....#####.#
....#..#...##..#.#.###..
.####...#..#.....#......
#..#.##..#..###.#.##....
#.####..#.####.#.#.###..
###.#.#...#.######.#..##
#.####....##..########.#
##..##.#...#...#.#.#.#..
...#..#..#.#.##..###.###
.#.#....#.##.#...###.##.
###.#...#..#.##.######..
.#.#.###.##.##.#..#.##..
.####.###.#...###.#..#.#
..#.#..#..#.#.#.####.###
#..####...#.#.#.###.###.
#####..#####...###....##
#.##..#..#...#..####...#
.#.###..##..##..####.##.
...###...##...#...#..###"""
x.count('#')

303

In [88]:
print(has_sea_monster(big_grid))

[(1, 26), (2, 44), (6, 14), (15, 58), (20, 81), (22, 4), (22, 50), (24, 31), (27, 75), (28, 63), (28, 86), (33, 19), (41, 25), (42, 40), (48, 14), (50, 90), (54, 56), (58, 67), (63, 1), (63, 78), (64, 36), (67, 51), (70, 41), (73, 8)]


In [89]:
print_grid(big_grid)

#...#.#...........##..#...##.............#.#....#.......#.......................................
#.##.#..........#........#.#..#....#.......#..#........##..##.#...#.#...#.#.#....#..#.....#...#.
.#....#...#..................##.....#.....#..#.####...#........#...###..####....###.##.....###.#
....#....#.#....#....#.#......##..##...###.#..#.......##...#....#.###.##.#.#####..#......##....#
###..#..#.#.....#.#.#..##....#...#......##.........#........#................#......###.........
.........#........#...#.#..###...###.#.###.......#.....##..#...#..#.#............#.##......#.#..
......#...#...#..#....##..#..#.##.##.##...##.##.#..#..##.....#...#......#..##....##.##.##....#..
....#.#.#......#.....#..#.................#........#..........#.#.#.#...#.##..##..###...........
.......#..#...#...#..#.#.#.........##..#.#.....#........##.....#...#.#.....##..##...#......#....
........#.#...#.#..........................#...#....#.......#........###.#..#.###..###.#..####..
#..#..#..#.#.......#.#.#...#.#

##..................#.....#........#...##....#.............#...#...##.#......#....##...##.......
.........##.............#..###..#.......#........##.#......#......#......##..#.......#..#.......


In [93]:
used_points = set()

def is_sea_monster(grid, x_off, y_off):
    for x in range(sea_monster_width):
        for y in range(sea_monster_height):
            if sea_monster[y][x] == '#' and grid[y_off+y][x_off+x] != '#':
                return False
    for x in range(sea_monster_width):
        for y in range(sea_monster_height):
            if sea_monster[y][x] == '#':
                used_points.add((x+x_off, y+y_off))
    return True

def has_sea_monster(grid):
    sea_monsters = [
        
    ]
    for x_start in range(gridwidth - sea_monster_width):
        for y_start in range(gridheight - sea_monster_height):
            if is_sea_monster(grid, x_start, y_start):
                sea_monsters.append((x_start, y_start))
    return sea_monsters

has_sea_monster(big_grid)
len(used_points)

360

In [94]:
sum_hash = 0
for row in big_grid:
    sum_hash += row.count('#')
sum_hash

2201

In [95]:
2201-360

1841